In [2]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# # models
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50

In [4]:
data = h5py.File("../dataset/data_sample.hdf5", "r")
print("data keys: " + str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [5]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = " + str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [6]:
def one_hot_encode(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    labels_digit = np.repeat(labels_digit, 48)
    labels_onehot = np.zeros((labels_digit.shape[0], 6))
    labels_onehot[np.arange(labels_digit.shape[0]), labels_digit] = 1
    return labels_onehot

In [7]:
def build_dataset(images, labels):
    
    train_indices = np.random.choice(np.arange(images.shape[0]), int( 0.7 * images.shape[0]))
    val_indices = list(set(np.arange(images.shape[0])) - set(train_indices))
    
    x = np.transpose(images, (0, 3, 1, 2))
    x = np.expand_dims(x, len(x.shape))
    y = one_hot_encode(labels).reshape(-1, 48, 6)
    
    return x, y

In [8]:
x, y = build_dataset(images, labels)

In [ ]:
## shuffle and row selection are
# very slow
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)
## maybe flatten -> select -> reshape back ??

In [11]:
def init_model(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48):
    
    intput_shape = (timestep, cnn_input_dim, cnn_input_dim, 1)
    
    model = Sequential()
    
    # CNN
    model.add(TimeDistributed(Conv2D(32, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), input_shape=intput_shape))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(Dense(cnn_output_dim, activation='relu')))
    
    # RNN
    # rnn_input_dim = cnn_output_dim
    # encode
    model.add(LSTM(rnn_hidden_dim, input_shape=(timestep, cnn_output_dim)))
    # repeat vector for timestep
    model.add(RepeatVector(timestep))
    # decode
    # TODO: do some research on stacked LSTM
    model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True))
    # add dense
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))
    
    return model

In [12]:
model = init_model()

In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
batch_size = 32
epochs = 20

model.fit(x[:500], y[:500],
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x[600:700], y[600:700]))

In [235]:
def train(trainset, testset, cnn, rnn, batch_size=128, num_epoch=10, lr=0.0001, 
          print_every=100, early_stop=True):
    pass